In [1]:
from decouple import config
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
import psycopg2
from sqlalchemy import create_engine

# Configura la conexión a PostgreSQL
user = config('DB_USER')
password = config('DB_PASSWORD')
host = config('DB_HOST')
database = config('DB_DATABASE')
table_name = 'cnv'
# Crea la conexión usando sqlalchemy
engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}')


sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()

Conexion exitosa con la Base de datos:irvin_hisminsa


In [5]:
ruta_datos = os.path.abspath("D:\Data\CNV")
archivos_excel = gb.glob(os.path.join(ruta_datos, '*.xlsx'))

# Inicializar un DataFrame vacío
df = pd.DataFrame()

for archivo in archivos_excel:
    print("Leyendo archivo:", archivo)
    # Leer la primera hoja del archivo Excel
    df_cnv = pd.read_excel(archivo, sheet_name=0)
    
    # Concatenar al DataFrame principal
    df = pd.concat([df, df_cnv], ignore_index=True)

# Imprimir información sobre el DataFrame resultante
print("Información del DataFrame combinado:")
print(df.info())

# Ver las primeras filas del DataFrame combinado
print(df.head())







<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Irvin\AppData\Local\Temp\ipykernel_16780\404052669.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta_datos = os.path.abspath("D:\Data\CNV")


Leyendo archivo: D:\Data\CNV\CNV_DOM_MADRE_CUSCO_060324.xlsx
Leyendo archivo: D:\Data\CNV\CNV_LUGARNACIDO_CUSCO_060324.xlsx
Información del DataFrame combinado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162143 entries, 0 to 162142
Data columns (total 53 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   NU_CNV                     162143 non-null  int64  
 1   Ubigeo_LugarNacido         162143 non-null  int64  
 2   DPTO_EESS                  162143 non-null  object 
 3   PROV_EESS                  162143 non-null  object 
 4   DIST_EESS                  162143 non-null  object 
 5   Ipress                     162143 non-null  int64  
 6   CO_LOCAL                   162143 non-null  int64  
 7   Nombre_EESS                162143 non-null  object 
 8   Diresa_Diris               162143 non-null  object 
 9   Institución                162143 non-null  object 
 10  Categoria                  162143 non-n

In [7]:
df.columns = df.columns.str.lower()
df['fe_nacido'] = pd.to_datetime(df['fe_nacido'], format='%Y%m%d')
df['fe_crea'] = pd.to_datetime(df['fe_crea'], format='%Y%m%d%H%M%S')
df['nu_doc_madre'] = df['nu_doc_madre'].str.strip()
df['nu_cnv'] = df['nu_cnv'].astype(str)
df_sin_duplicados = df.drop_duplicates(subset=['tipo_doc_madre','fe_nacido', 'nu_doc_madre'])
#df_sin_duplicados = df.drop_duplicates(subset=['nu_cnv','tipo_doc_madre','fe_nacido', 'nu_doc_madre'])

#df.head()

df_sin_duplicados.head(5)



,nu_cnv,ubigeo_lugarnacido,dpto_eess,prov_eess,dist_eess,ipress,co_local,nombre_eess,diresa_diris,institución,...,ubigeo_dom_madre,edad_madre,profesion_prof,colegiat_prof,tip_docum_prof,nu_doc_prof,prim_ape_prof,seg_ape_prof,prenombres_prof,fe_crea
0,91666906,80108,CUSCO,CUSCO,WANCHAQ,10041,10041,ADOLFO GUEVARA VELASCO,CUSCO,ESSALUD,...,80108.0,31,2,2905.0,1,23962027.0,MONTEAGUDO,RUIZ CARO,JOHANA MIRIAN ...,2020-01-05 16:56:21
1,91666922,80801,CUSCO,ESPINAR,ESPINAR,7135,7135,ESPINAR,CUSCO,GOBIERNO REGIONAL,...,80801.0,21,11,46374.0,1,30488040.0,ESPINOZA,QUISPE,ELIAS WILMAN ...,2020-01-05 17:12:33
2,91667058,80108,CUSCO,CUSCO,WANCHAQ,11339,11339,SAN JOSE,CUSCO,PRIVADO,...,80101.0,28,11,23454.0,1,23990034.0,VIZCARRA,LOAYZA,JUSTO GERMAN ...,2020-01-05 19:26:40
3,91669838,80901,CUSCO,LA CONVENCION,SANTA ANA,2421,2421,QUILLABAMBA,CUSCO,GOBIERNO REGIONAL,...,80913.0,29,2,8550.0,1,2412254.0,CHURA,COQUE?A,RUTH ENEIDA ...,2020-01-07 12:13:57
4,91669861,80106,CUSCO,CUSCO,SANTIAGO,2305,2305,ANTONIO LORENA DEL CUSCO,CUSCO,GOBIERNO REGIONAL,...,80106.0,24,16,60193.0,1,40213711.0,AIMA,LINARES,ROCIO ...,2020-01-07 12:23:40


In [8]:
# Inserta el DataFrame en la tabla de PostgreSQL


df_sin_duplicados.to_sql(table_name, engine,schema='maestros', if_exists='replace', index=False)

# Cierra la conexión
engine.dispose()